### 1. Import packages

In [1]:
import numpy as np
import pandas as pd
import random
from IPython import display
from collections import namedtuple, deque
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cpu")
# use cpu run
import gym

### 2. Helper functions

In [2]:
def dict2array(state):
    new_state = []
    for key  in state.keys():
        if key != 'sw':
            new_state.append(state[key])
        else:
            new_state += list(state['sw'])        
    state = np.asarray(new_state)
    
    return state

### 3. Initialize the environment

In [3]:
env_args = {
    'run_dssat_location': '/opt/dssat_pdi/run_dssat',  # assuming (modified) DSSAT has been installed in /opt/dssat_pdi
    'log_saving_path': './logs/dssat-pdi.log',  # if you want to save DSSAT outputs for inspection
    # 'mode': 'irrigation',  # you can choose one of those 3 modes
    # 'mode': 'fertilization',
    'mode': 'all',
    'seed': 123456,
    'random_weather': False,  # if you want stochastic weather
}
env = gym.make('gym_dssat_pdi:GymDssatPdi-v0', **env_args)
print('Observation:',env.observation,)
print(len(env.observation),len(env.observation['sw']))
ram_dimensions = len(env.observation) + 8
nb_actions = 11
print('\nRam information received from DASSAT will has %d dimensions.' % ram_dimensions)
print('There are %d possible actions at each step.' % nb_actions)
print('Discrete?',type(gym.spaces)== gym.spaces.Discrete)
# observation has 27 elements, 9 values in soil water
# state size = 27+8=35 dimension
# how to defind nb_action? why is 200?
state = env.reset()
state=dict2array(state)
print(state)
#index of tleachd = 25
#index of topwt = 29

Observation: {'cleach': 0.0, 'cnox': 0.0, 'cumsumfert': 0.0, 'dap': 0, 'dtt': 0.0, 'es': 0.0, 'grnwt': 0.0, 'istage': 7, 'nstres': 0.0, 'pcngrn': 0.0, 'pltpop': 7.199999809265137, 'rain': 0.0, 'rtdep': 0.0, 'runoff': 0.0, 'srad': 13.300000190734863, 'sw': array([0.086     , 0.086     , 0.086     , 0.086     , 0.086     ,
       0.076     , 0.076     , 0.13      , 0.25799999]), 'swfac': 0.0, 'tleachd': 0.0, 'tmax': 22.200000762939453, 'tmin': 3.299999952316284, 'tnoxd': 0.0, 'topwt': 0.0, 'trnu': 0.0, 'vstage': 0.0, 'wtdep': 0.0, 'wtnup': 0.0, 'xlai': 0.0}
27 9

Ram information received from DASSAT will has 35 dimensions.
There are 11 possible actions at each step.
Discrete? False
[ 0.          0.          0.          0.          0.          0.
  0.          7.          0.          0.          7.19999981  0.
  0.          0.         13.30000019  0.086       0.086       0.086
  0.086       0.086       0.076       0.076       0.13        0.25799999
  0.          0.         22.20000076  3.

### 4. Define the network

In [4]:
class QNetwork(nn.Module):
    """Agent (Policy) Model."""
    # given a state of 35 dim, Qnetwork will return 200 values for each possible action  

    def __init__(self, state_size, action_size, fc1_units=256):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            fc1_units (int): Number of nodes in first hidden layer
            why is it 256? randomly?
        """
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, action_size)
        # set a nn with 1 layer
        
    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        #Applies the rectified linear unit function element-wise. max(0,x)
        return self.fc2(x)

### 5. Define the Replay Buffer

In [5]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size):
        """Initialize a ReplayBuffer object.
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            how many samples stored in buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        # store [s,a,r,s',done] for each sample in buffer
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
#         action = dict2array(action)
#         print('transfored actions:',action)
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)
#         for e in experiences:
# #             print(e)
# #             print(e.state.shape, e.next_state.shape)
#             break
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        # vstack is add rows together as a single matrix
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        #get all the states, actions, rewards, next_states, dones for all elements in this sample batch, each as a single matrix
        #device here is cpu?
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

### 6. Define the Agent

In [6]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 60                # for update of target network parameters
LR = 1e-5               # learning rate 
UPDATE_EVERY = 4        # how often to update the network

### Reward function

In [7]:
class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size):
        """Initialize an Agent object.
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
        """
        self.state_size = state_size
        self.action_size = action_size

        # Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        # add current (s,r,a,s') and sample a batch and learn if update_every
        self.memory.add(state, action, reward, next_state, done)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step += 1
        if self.t_step%UPDATE_EVERY == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()
#         Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        # return the action index with the largest value
        else:
            return random.choice(np.arange(self.action_size))
            #return one action randomly with possibility eps

#         Epsilon-greedy action selection
#         if random.random() > eps:
#             return np.argmax(action_values.cpu().data.numpy())
#         else:
#             return random.choice(np.arange(self.action_size))
# #         return action_values.cpu().data.numpy()


    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.
        Params
        ======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # detach for diemnsion correctness
        # GET largest value of self.qnetwork_target(next_states)
#         print(Q_targets_next)
        # Compute Q targets for current states 
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        # 0 if done

        # Get expected Q values from local model
#         print('out',self.qnetwork_local(states).shape)
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        #Why need to gather? actions are indexs?

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        # after this, the parameter of local network will change based on gradient descent
        for param in self.qnetwork_local.parameters():
            param.grad.data.clamp_(-1, 1)
        # stabilize traning to keep grad between (-1,1)
        self.optimizer.step()
        #固定用法, change learning rate

        #Update target network weights every TAU learning steps (so every TAU*UPDATE_EVERY t_step)
        if self.t_step%(TAU*UPDATE_EVERY)==0:
            self.update_target_net(self.qnetwork_local, self.qnetwork_target)                     

    def update_target_net(self, local_model, target_model):
        """Soft update model parameters.
        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
        """
        target_model.load_state_dict(local_model.state_dict())

In [8]:
def get_reward(state, action, next_state, done, k, action_number):
    #if done, return Yield (topwt) - k*cost
    if action_number > 3:
        cost = 2000*(action_number-3) + action
        # cost = punishment of actions + current action
    else:
        cost = action
    if done:
        return state[29]- k*action
    #if done, return Yield (topwt) - k*cost
    else:
        return -k*(cost-next_state[25])
    # otherwise, return -k*(cost+leaching)

### DQN for nitrogen management

In [9]:
agent = Agent(state_size=ram_dimensions, action_size=nb_actions)

In [10]:
def dqn(n_episodes=2000, max_t=500, eps_start=1.0, eps_end=0.01, eps_decay=0.997, solved=495, optimize = True):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    start = time.time()                # Start time
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    list_eps = []
    eps = eps_start                    # initialize epsilon
    action_list=[]
    action_num_list=[]
    action_num=0
    for i_episode in range(1, n_episodes+1):
        # Reset env and score at the beginning of episode
        action_not_zero=0
        state = env.reset() # reset the environment and get current state
        state = dict2array(state)
        score = 0 # initialize the score
        for t in range(max_t):
            action1 = agent.act(state, eps)*20 if optimize else 100
            if action1 != 0:
                action_num=action_num+1
            if i_episode == 1000:
                action_list.append(action1)
            #action_n++
            # for debug
            action = {
                    'anfer': action1,  # if mode == fertilization or mode == all ; nitrogen to fertilize in kg/ha
                    'amir': 0,  # if mode == irrigation or mode == all ; water to irrigate in L/ha
            }
            next_state, reward, done, _ = env.step(action)      # send the action to the environment\
            if done:
                next_state = state
                reward = get_reward(state, action1, next_state, done, 0.5, action_num)
                #set next_state as current state for type consistency
                agent.step(state, action['anfer'], reward, next_state, done)
                scores += reward
                print('episode is %d .' % i_episode)
                print('action_num is %d .' % action_num)
                print('score is %d .' % score)
                print('')
                break
            next_state=dict2array(next_state)
            reward = get_reward(state, action1, next_state, done, 0.5, action_num)
            agent.step(state, action['anfer'], reward, next_state, done)
            state = next_state
            score += reward
        if i_episode == 1000:
            print('action_num is %d .' % action_num)
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        list_eps.append(eps)
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        
            
        
        if i_episode % 100 == 0:
            print('\rEpisode {}/{} \tAverage Score: {:.2f}'.format(i_episode, n_episodes, np.mean(scores_window)))
            print('Epsilon: {}'.format(eps))
            
        if np.mean(scores_window)>solved:
            print('Game Solved after {} episodes'.format(i_episode))
            break

    time_elapsed = time.time() - start
    print("Time Elapse: {:.2f} seconds".format(time_elapsed))
    
    return scores, list_eps, action_list, action_num_list

In [19]:
scores, list_eps, action_list, action_num_list= dqn(n_episodes=1000)

Qtarget is
tensor([[-1.4944e+04],
        [-1.7929e+04],
        [-8.9450e+03],
        [-5.1657e+04],
        [-1.0971e+04],
        [-2.7900e+04],
        [-5.9589e+04],
        [-4.1738e+04],
        [-7.0171e+03],
        [-3.8767e+04],
        [-3.4767e+04],
        [-2.2924e+04],
        [-3.2782e+04],
        [-5.4703e+04],
        [-1.7761e+01],
        [-4.0150e+03],
        [-2.6884e+04],
        [-4.5716e+04],
        [-2.8796e+04],
        [-2.0765e+03],
        [-4.9734e+04],
        [-5.7654e+04],
        [-3.5836e+04],
        [-3.3803e+04],
        [-6.0646e+04],
        [-2.9849e+04],
        [-1.9889e+04],
        [-1.3108e+01],
        [-1.6922e+04],
        [-3.0117e+03],
        [-1.8941e+04],
        [-5.2677e+04],
        [-3.1776e+04],
        [-1.0392e+03],
        [-4.7691e+04],
        [-2.0877e+04],
        [-3.1836e+04],
        [-5.0085e+03],
        [-2.4915e+04],
        [-4.0781e+04],
        [-2.1893e+04],
        [-2.3915e+04],
        [-1.1974e+04],


RuntimeError: index 80 is out of bounds for dimension 1 with size 11

### Given a constant input: reward = 19847

In [12]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(scores)
plt.yscale('symlog')
plt.title("Nitrogen Management")
plt.xlabel("Traing Steps")
plt.ylabel("Rewards")
plt.savefig("reward_0 water with exponential penalty learning_false.png")

In [12]:
plt.figure()
print(action_list)
plt.plot(action_list)
plt.title("Naction of last episode")
plt.xlabel("Days")
plt.ylabel("Nitrogen input")
plt.savefig("action_20 water with linear penalty learning_false.png")

[0, 0, 0, 0, 0, 0, 43, 0, 0, 0, 169, 0, 0, 0, 0, 0, 0, 0, 0, 78, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 198, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 67, 0, 0, 68, 0, 0, 150, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 188, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 105, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 119, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 127, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 100]


In [16]:
print(np.average(action_num_list))

8.52


In [28]:
score1=scores[-10:]
plt.plot(score1)
plt.title("Nitrogen Management")
plt.xlabel("Traing Steps")
plt.ylabel("Rewards")
plt.savefig("reward_partial_0 water with exponential penalty learning_false.png")

In [33]:
print(scores[-10:])

[-11112234.922243932, -1136.1068184592432, -1084.0878253100661, -996.9282381844116, -1112060.2553732428, -1112161.5164292145, -112175.58849308883, -1066.5072541128247, -2118.796046328218, -12061.030217774274]
